In [1]:
import openseespy.opensees as ops
import numpy as np
import opsvis as opsv
import matplotlib.pyplot as plt
import os
from ipywidgets import interact, FloatSlider, IntSlider

# Units
m = 1
N = 1
kN = 1000*N
Pa = 1
mm = 0.001 * m

inches = 25.4 * mm
ft = 12 * inches
kip = 4.45 * 10**3 * N
ksi = 6.89 * 10**6 * Pa

# Initial portal frame dimensions
width = 30 * ft
height = 12 * ft

# Initial column dimensions
colWidth = 15 * inches
colDepth = 24 * inches

# Initial steel reinforcement
As = (0.60 * inches)**2  # area of no. 7 bars
fy = 60.0  # Yield stress
E = 30000.0  # Young's modulus

# Beam properties
A_beam = 360.0
E_beam = 4030.0
Iz_beam = 8640.0

# Set deflection tolerance
tol = 1e-6

# Create file path for data output
if not os.path.exists('Data'):
    os.mkdir('Data')

def create_model(col_length, beam_length, col_dims, y_core_div, z_core_div, top_bottom_div, left_right_div, rebar_area, cover, load_mag):
    ops.wipe()
    ops.model('basic', '-ndm', 2, '-ndf', 3)
    
    # Define nodes
    ops.node(1, 0.0, 0.0)
    ops.node(2, beam_length, 0.0)
    ops.node(3, 0.0, col_length)
    ops.node(4, beam_length, col_length)
    
    # Fix supports at base of columns
    ops.fix(1, 1, 1, 1)
    ops.fix(2, 1, 1, 1)
    
    # Define materials for nonlinear columns
    ops.uniaxialMaterial('Concrete01', 1, -6.0, -0.004, -5.0, -0.014)
    ops.uniaxialMaterial('Concrete01', 2, -5.0, -0.002, 0.0, -0.006)
    ops.uniaxialMaterial('Steel01', 3, fy, E, 0.01)
    
    col_width, col_height = col_dims
    y1 = col_height / 2 
    z1 = col_width / 2
    fib_sec_1 = [['section', 'Fiber', 1, '-GJ', 1.0e6],
                 ['patch', 'rect', 1, y_core_div, z_core_div, cover-y1, cover-z1, y1-cover, z1-cover],  # core concrete
                 ['patch', 'rect', 2, left_right_div, 1, -y1, z1-cover, y1, z1],  # left cover
                 ['patch', 'rect', 2, left_right_div, 1, -y1, -z1, y1, cover-z1],  # right cover
                 ['patch', 'rect', 2, top_bottom_div, 1, -y1, cover-z1, cover-y1, z1-cover],  # bottom cover
                 ['patch', 'rect', 2, top_bottom_div, 1, y1-cover, cover-z1, y1, z1-cover],  # top cover
                 ['layer','straight', 3, 3, rebar_area, y1-cover, z1-cover, y1-cover, cover-z1],  # Left layer of steel reinforcement
                 ['layer','straight', 3, 2, rebar_area, 0.0, z1-cover, 0.0, cover-z1],  # Middle layer
                 ['layer','straight', 3, 3, rebar_area, cover-y1, z1-cover, cover-y1, cover-z1]  # Right layer
                 ]     
    opsv.fib_sec_list_to_cmds(fib_sec_1)

    # Define column elements
    ops.geomTransf('PDelta', 1)
    ops.beamIntegration('Lobatto', 1, 1, 5)
    ops.element('forceBeamColumn', 1, 1, 3, 1, 1)
    ops.element('forceBeamColumn', 2, 2, 4, 1, 1)
    
    # Define beam element
    ops.geomTransf('Linear', 2)
    ops.element('elasticBeamColumn', 3, 3, 4, A_beam, E_beam, Iz_beam, 2)
    
    # Define gravity loads
    load_mag = load_mag * kN
    ops.timeSeries('Linear', 1)
    ops.pattern('Plain', 1, 1)
    ops.load(3, 0.0, -load_mag, 0.0)
    ops.load(4, 0.0, -load_mag, 0.0)
    
    # Perform analysis
    ops.system('BandGeneral')
    ops.numberer('RCM')
    ops.constraints('Transformation')
    ops.integrator('LoadControl', 0.1)
    ops.algorithm('Newton')
    ops.analysis('Static')
    ops.analyze(10)
    
    # Plot the model
    opsv.plot_model()
    plt.title('Portal Frame Model nodes and elements')
    plt.xlabel('Portal frame width (m)')
    plt.ylabel('Portal frame height (m)')
    plt.show()
    
    # Plot deformed shape
    
    n3x = ops.nodeDisp(3, 1)
    n3y = ops.nodeDisp(3, 2)

    n4x = ops.nodeDisp(4, 1)
    n4y = ops.nodeDisp(4, 2)

    opsv.plot_defo(interpFlag=0)
    plt.title(f'Deformed shape\nNode 3 Displacement: x = {n3x:.4f} m, y = {n3y:.4f} m\nNode 4 Displacement: x = {n4x:.4f} m, y = {n4y:.4f} m')
    plt.xlabel('Portal frame width (m)')
    plt.ylabel('Portal frame height (m)')
    plt.show()
  

def plot_fiber_section(col_width, col_height, y_core_div, z_core_div, top_bottom_div, left_right_div, rebar_area, cover):
    y1 = col_height / 2 
    z1 = col_width / 2
    fib_sec_1 = [['section', 'Fiber', 1, '-GJ', 1.0e6],
                 ['patch', 'rect', 1, y_core_div, z_core_div, cover-y1, cover-z1, y1-cover, z1-cover],  # core concrete
                 ['patch', 'rect', 2, left_right_div, 1, -y1, z1-cover, y1, z1],  # left cover
                 ['patch', 'rect', 2, left_right_div, 1, -y1, -z1, y1, cover-z1],  # right cover
                 ['patch', 'rect', 2, top_bottom_div, 1, -y1, cover-z1, cover-y1, z1-cover],  # bottom cover
                 ['patch', 'rect', 2, top_bottom_div, 1, y1-cover, cover-z1, y1, z1-cover],  # top cover
                 ['layer','straight', 3, 3, rebar_area, y1-cover, z1-cover, y1-cover, cover-z1],  # Left layer of steel reinforcement
                 ['layer','straight', 3, 2, rebar_area, 0.0, z1-cover, 0.0, cover-z1],  # Middle layer
                 ['layer','straight', 3, 3, rebar_area, cover-y1, z1-cover, cover-y1, cover-z1]  # Right layer
                 ]            
    matcolor = ['r', 'lightgrey']
    opsv.plot_fiber_section(fib_sec_1, matcolor=matcolor)
    plt.axis('equal')
    plt.title('RC Column Cross-Section')
    plt.xlabel('Column width (mm)')
    plt.ylabel('Column height (mm)')
    plt.show()

def interactive_model(col_length, beam_length, col_width, col_height, y_core_div, z_core_div, top_bottom_div, left_right_div, rebar_area, cover, load_mag):
    col_dims = (col_width, col_height)
    create_model(col_length, beam_length, col_dims, y_core_div, z_core_div, top_bottom_div, left_right_div, rebar_area, cover, load_mag)
    plot_fiber_section(col_width, col_height, y_core_div, z_core_div, top_bottom_div, left_right_div, rebar_area, cover)

interact(interactive_model,
         col_length=FloatSlider(min=1, max=5, step=0.1, value=height, description='Column Length (m):', style={'description_width': 'initial'}),
         beam_length=FloatSlider(min=1, max=20, step=0.1, value=width, description='Beam Length (m):', style={'description_width': 'initial'}),
         col_width=FloatSlider(min=200, max=500, step=0.1, value=300, description='Column Width (mm):', style={'description_width': 'initial'}),
         col_height=FloatSlider(min=400, max=700, step=0.1, value=500, description='Column Height (mm):', style={'description_width': 'initial'}),
         z_core_div=IntSlider(min=1, max=3, step=1, value=1, description='Core Divisions (z-axis):', style={'description_width': 'initial'}),
         y_core_div=IntSlider(min=1, max=20, step=1, value=10, description='Core Divisions (y-axis):', style={'description_width': 'initial'}),
         top_bottom_div=IntSlider(min=1, max=3, step=1, value=1, description='Top and Bottom Cover Divisions:', style={'description_width': 'initial'}),
         left_right_div=IntSlider(min=1, max=10, step=1, value=5, description='Left and right Cover Divisions:', style={'description_width': 'initial'}),
         rebar_area=FloatSlider(min=100, max=500, step=1, value=250, description='Rebar area (mm^2):', style={'description_width': 'initial'}),
         cover=FloatSlider(min=25, max=100, step=1, value=50, description='Cover thickness (mm):', style={'description_width': 'initial'}),
         load_mag=FloatSlider(min=10, max=1000, step=1, value=100, description='load magnitude (kN):', style={'description_width': 'initial'}))
         
         
        

interactive(children=(FloatSlider(value=3.6575999999999995, description='Column Length (m):', max=5.0, min=1.0…

<function __main__.interactive_model(col_length, beam_length, col_width, col_height, y_core_div, z_core_div, top_bottom_div, left_right_div, rebar_area, cover, load_mag)>